# Introducción 

# Objetivos 

El objetivo general de la prueba `recommender_system_test` es evaluar el impacto de un sistema de recomendaciones mejorado en el comportamiento de los usuarios en términos de conversión a través del embudo de pago. La prueba involucra dos grupos: el grupo de control (A) y el grupo experimental con el nuevo embudo de pago (B). La prueba se lanzó el 7 de diciembre de 2020, dejó de aceptar nuevos usuarios el 21 de diciembre de 2020 y finalizó el 1 de enero de 2021. La audiencia de la prueba consiste en el 15% de los nuevos usuarios de la región de la UE, con un número previsto de 6,000 participantes. El propósito de la prueba es medir si, dentro de los 14 días posteriores a la inscripción, los usuarios muestran una mejor conversión en vistas de la página del producto (`product_page`), en la acción de agregar artículos al carrito de compras (`product_cart`) y en las compras (`purchase`). Se espera que en cada etapa del embudo (`product_page → product_cart → purchase`), haya al menos un aumento del 10%. Los datos de la prueba deben descargarse, comprobarse para asegurar que la prueba se realizó correctamente, y analizarse para evaluar los resultados obtenidos.


# Descomposición de la tarea

* ...

# Carga de librerías 

In [1]:
# Carga de librerías 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go

# Carga de datos

In [2]:
# Carga de datasets desde carpeta local
data_path = os.path.join("..", "datasets")

dataset1_path = os.path.join(data_path, "ab_project_marketing_events_us.csv")
dataset2_path = os.path.join(data_path, "final_ab_events_upd_us.csv")
dataset3_path = os.path.join(data_path, "final_ab_new_users_upd_us.csv")
dataset4_path = os.path.join(data_path, "final_ab_participants_upd_us.csv")

# el calendario de eventos de marketing para 2020
ab_project_marketing_data = pd.read_csv(dataset1_path)

# todos los usuarios que se registraron en línea desde 7 al 21 de diciembre
final_ab_new_events_data = pd.read_csv(dataset3_path)

# todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021
final_ab_events_data = pd.read_csv(dataset2_path)

# tabla con los datos de los participantes de la prueba
final_ab_participants_data = pd.read_csv(dataset4_path)

In [3]:
# Carga de datos en jupyter de platforma

"""ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")
final_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")
final_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")
final_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")"""

'ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")\nfinal_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")\nfinal_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")\nfinal_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")'

## Exploración inicial de datos

### Revisión de dataset ab_project_marketing_data

In [4]:
# Revisión de datos ab_project_data
print('- Información sobre dataset -')
print(ab_project_marketing_data.head(5))
print()
print(ab_project_marketing_data.info())
print()
print('- Datos ausentes -')
print(ab_project_marketing_data.isna().sum())
print()
print('- Valores duplicados -')
print(ab_project_marketing_data.duplicated().sum())

- Información sobre dataset -
                           name                   regions    start_dt  \
0      Christmas&New Year Promo             EU, N.America  2020-12-25   
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2        St. Patric's Day Promo             EU, N.America  2020-03-17   
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4             4th of July Promo                 N.America  2020-07-04   

    finish_dt  
0  2021-01-03  
1  2020-02-16  
2  2020-03-19  
3  2020-04-19  
4  2020-07-11  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes
None

- Datos ausentes -
name         0
regions      

- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

In [5]:
# Cambio de formato de columna start_dt
ab_project_marketing_data['start_dt'] = pd.to_datetime(ab_project_marketing_data['start_dt'])
ab_project_marketing_data['finish_dt'] = pd.to_datetime(ab_project_marketing_data['finish_dt'])

### Revisión de dataset final_ab_events_data

In [6]:
# Revisión de dataset final_ab_events_data
print('- Información sobre dataset -')
print(final_ab_events_data.head())
print()
print(final_ab_events_data.info())
print()
print('- Datos ausentes -')
print(final_ab_events_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_events_data.duplicated().sum())

- Información sobre dataset -
            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB
None

- Datos ausentes -
user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64

- Valores duplicados -
0


- `user_id`
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos `purchase`)

In [7]:
# Cambio de formato de columna event_date
final_ab_events_data['event_dt'] = pd.to_datetime(final_ab_events_data['event_dt'])

In [8]:
# Revisión de datos en la columna 'details'
final_ab_events_data['details'].sample(10)

268988     NaN
53040     4.99
91207      NaN
65551      NaN
14284     4.99
94451      NaN
166242     NaN
315535     NaN
393066     NaN
307891     NaN
Name: details, dtype: float64

In [9]:
# Revisión de datos que contienen 'NaN' en la columna 'details'
no_purchase = final_ab_events_data[final_ab_events_data['details'].isna()]
no_purchase.sample(10)

# Se aplica reemplazo de valores NaN por 0 
final_ab_events_data.fillna({'details':0}, inplace = True)
print(final_ab_events_data.sample(10))

                 user_id            event_dt    event_name  details
317510  AE933822F3F98847 2020-12-17 19:54:19         login     0.00
270879  4625D0FC513790BC 2020-12-12 18:05:41         login     0.00
48635   EB7EE27F44A551D5 2020-12-23 08:36:22      purchase     4.99
57258   90543C331D6A6EE7 2020-12-27 05:27:35      purchase     4.99
143543  7D95ED62ED968215 2020-12-12 03:39:42  product_page     0.00
279140  28E22C47E27E2A19 2020-12-13 18:02:14         login     0.00
197010  BFDD31D463F35AAE 2020-12-20 23:49:49  product_page     0.00
317247  FB54FA38726D502D 2020-12-17 14:26:34         login     0.00
407644  10549DF81E54149F 2020-12-26 15:22:31         login     0.00
140768  2EB435DCFA729990 2020-12-12 11:52:53  product_page     0.00


In [10]:
# Verificación de los valores de eventos
final_ab_events_data['event_name'].value_counts()

event_name
login           182465
product_page    120862
purchase         60314
product_cart     60120
Name: count, dtype: int64

Columna 'details' contiene información numérica y NaN, la documentación del dataset indica que tiene la cantidad en USD para cuando el evento es 'purchase'. Se infiere que los datos NaN son 0, ya que no representan una transacción concretada. 
Se observa que hay más valores de `purchase` que de `product_cart`, lo que puede sugerir que se evita esta última etapa y se puede comprar con algo parecido a un botón 'compra rápida'

### Revisión de dataset final_ab_new_events_data

In [11]:
# Revisión de datos en dataset final_ab_new_events_data
print('- Información sobre dataset -')
print(final_ab_new_events_data.sample(10))
print()
print(final_ab_new_events_data.info())
print()
print('- Datos ausentes -')
print(final_ab_new_events_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_new_events_data.duplicated().sum())

- Información sobre dataset -
                user_id  first_date     region   device
55525  AD13ECB32CD5A112  2020-12-20       APAC  Android
30361  F2B572D9737DE6ED  2020-12-23         EU  Android
21585  40C2B7BF1CE07A2C  2020-12-15  N.America  Android
7431   4BF00AA98745F2B4  2020-12-14         EU       PC
17924  7D535874195B6A6F  2020-12-08         EU   iPhone
40871  795900E246C0B027  2020-12-18  N.America   iPhone
5065   8691349815230E5F  2020-12-07         EU   iPhone
45861  E5AD148BE741E68E  2020-12-12         EU  Android
4111   9E7A65A9C6A8CE0E  2020-12-07         EU  Android
23266  41379F437C739C0B  2020-12-22         EU  Android

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtype

- `user_id`
- `first_date`: fecha de inscripción
- `region`
- `device`: dispositivo utilizado para la inscripción

In [12]:
# Se cambia el formato de la columna 'first_date' 
final_ab_new_events_data['first_date'] = pd.to_datetime(final_ab_new_events_data['first_date'])


No contiene datos nulos y el único cambio realizado fue el formato de fecha en la columna 'first_date'

### Revisión del dataset final_ab_participants_data

In [13]:
# Revisión del dataset 

print('- Información sobre dataset -')
print(final_ab_participants_data.head())
print()
print(final_ab_participants_data.info())
print()
print('- Datos ausentes -')
print(final_ab_participants_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_participants_data.duplicated().sum())

- Información sobre dataset -
            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None

- Datos ausentes -
user_id    0
group      0
ab_test    0
dtype: int64

- Valores duplicados -
0


- `user_id`
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

# Segmentación de datos para realizar test A/B

Se requieren los datos que cumplan con las siguientes condiciones:
* Usuarios que se hayan inscrito entre 7 (inicio de la prueba) y 18 de diciembre (14 días antes de la fecha de finalización 01-01-2021). 
* Que pertenezcan a la región de UE
* Valor de la columna 'ab_test' = "recommender_system_test"
* Nombre del evento por usuario.
* El tiempo desde la inscripción del usuario hasta fecha de eventos este dentro de 14 días. 

In [24]:
# Generación de dataset de trabajo con las columnas requeridas

wdataset1 = pd.merge(final_ab_participants_data, final_ab_new_events_data, on='user_id', how = 'left')
wdataset2 = pd.merge(wdataset1, final_ab_events_data, on = 'user_id', how = 'left')

# Generación de columna con día del evento 
wdataset2['event_date'] = wdataset2['event_dt'].dt.date
wdataset2

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,event_date
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99,2020-12-07
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,0.00,2020-12-07
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,0.00,2020-12-07
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,login,0.00,2020-12-07
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone,2020-12-20 15:46:06,product_page,0.00,2020-12-20
...,...,...,...,...,...,...,...,...,...,...
102833,63FBE257B05F2245,A,interface_eu_test,2020-12-20,EU,Android,2020-12-23 13:03:17,login,0.00,2020-12-23
102834,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-14 19:12:49,login,0.00,2020-12-14
102835,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-15 01:50:22,login,0.00,2020-12-15
102836,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-16 19:27:36,login,0.00,2020-12-16


Data set de trabajo contiene los grupos, ab_test, fecha de inscripción, región, dispositivo, fecha del evento, nombre del evento, y detalles 

Data set contiene 102,833 registros.

In [23]:
wdataset2.groupby(['ab_test','group']).agg({'user_id':'nunique'})


user_id
ab_test                 group         
interface_eu_test       A         5467
                        B         5383
recommender_system_test A         2747
                        B          928

In [25]:
wdataset2['event_name'].value_counts()

event_name
login           44705
product_page    28961
purchase        14854
product_cart    14318
Name: count, dtype: int64